# 01. Import Libaray

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
from keras.utils import np_utils,to_categorical
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D,Conv1D,Conv2D
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Conv2D, Dense, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.layers import LSTM, TimeDistributed
from keras.layers import LeakyReLU

In [4]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import cv2

In [5]:
from sklearn import metrics

In [6]:
test_x = np.load('./test_x.npy')
test_y = np.load('./test_y.npy')
train_x = np.load('./train_x.npy')
train_y = np.load('./train_y.npy')

In [7]:
smote_tomek_train_x = np.load('./smote_tomek_train_x.npy')
smote_tomek_train_y = np.load('./smote_tomek_train_y.npy')

smote_tomek_test_x = np.load('./smote_tomek_test_x.npy')
smote_tomek_test_y = np.load('./smote_tomek_test_y.npy')

In [8]:
from sklearn.metrics import classification_report

In [9]:
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1]*train_x.shape[2]*train_x.shape[3]))
test_x = test_x.reshape((test_x.shape[0], test_x.shape[1]*test_x.shape[2]*test_x.shape[3]))

In [10]:
smote_tomek_train_x = smote_tomek_train_x.reshape((smote_tomek_train_x.shape[0], smote_tomek_train_x.shape[1]*smote_tomek_train_x.shape[2]*smote_tomek_train_x.shape[3]))
smote_tomek_test_x = smote_tomek_test_x.reshape((smote_tomek_test_x.shape[0], smote_tomek_test_x.shape[1]*smote_tomek_test_x.shape[2]*smote_tomek_test_x.shape[3]))

# Random Forest

In [11]:
# 일반 데이터
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=100, random_state=10)
clf.fit(train_x, train_y)

y_true = test_y
y_pred = clf.predict(test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.61      0.13      0.22       290
       happy       0.64      0.93      0.76       820
         sad       0.51      0.07      0.12       276
     neutral       0.53      0.63      0.57       507

    accuracy                           0.60      1893
   macro avg       0.57      0.44      0.42      1893
weighted avg       0.59      0.60      0.53      1893

0.6171374346840186


In [12]:
# smote_tomek 데이터
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=100, random_state=10)
clf.fit(smote_tomek_train_x, smote_tomek_train_y)

y_true = smote_tomek_test_y
y_pred = clf.predict(smote_tomek_test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.48      0.26      0.34       290
       happy       0.69      0.89      0.78       820
         sad       0.39      0.16      0.23       276
     neutral       0.55      0.61      0.58       507

    accuracy                           0.61      1893
   macro avg       0.53      0.48      0.48      1893
weighted avg       0.58      0.61      0.58      1893

0.5929100230342467


# Decision Tree

In [13]:
# 일반 데이터
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(train_x, train_y)

y_true = test_y
y_pred = clf.predict(test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.20      0.22      0.21       290
       happy       0.62      0.57      0.59       820
         sad       0.20      0.21      0.20       276
     neutral       0.40      0.41      0.40       507

    accuracy                           0.42      1893
   macro avg       0.35      0.35      0.35      1893
weighted avg       0.43      0.42      0.43      1893

0.5394158981115503


In [14]:
# smote_tomek 데이터
from sklearn.ensemble import RandomForestClassifier

clf = tree.DecisionTreeClassifier()
clf.fit(smote_tomek_train_x, smote_tomek_train_y)

y_true = smote_tomek_test_y
y_pred = clf.predict(smote_tomek_test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.24      0.28      0.26       290
       happy       0.65      0.57      0.61       820
         sad       0.27      0.32      0.29       276
     neutral       0.39      0.38      0.39       507

    accuracy                           0.44      1893
   macro avg       0.39      0.39      0.39      1893
weighted avg       0.46      0.44      0.45      1893

0.5413843402973838


# Support Vector Machine

In [15]:
# 일반 데이터
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_x, train_y)

y_true = test_y
y_pred = clf.predict(test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.58      0.27      0.37       290
       happy       0.73      0.88      0.80       820
         sad       0.32      0.09      0.14       276
     neutral       0.53      0.72      0.61       507

    accuracy                           0.63      1893
   macro avg       0.54      0.49      0.48      1893
weighted avg       0.59      0.63      0.59      1893

0.6087270217705001


In [16]:
# smote_tomek 데이터
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(smote_tomek_train_x, smote_tomek_train_y)

y_true = smote_tomek_test_y
y_pred = clf.predict(smote_tomek_test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.49      0.37      0.42       290
       happy       0.76      0.85      0.80       820
         sad       0.39      0.27      0.32       276
     neutral       0.57      0.64      0.61       507

    accuracy                           0.64      1893
   macro avg       0.55      0.53      0.54      1893
weighted avg       0.62      0.64      0.62      1893

0.6076156417771325


# Naive Bayies

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(train_x, train_y)

y_true = test_y
y_pred = gnb.predict(test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.28      0.35      0.31       290
       happy       0.55      0.64      0.59       820
         sad       0.26      0.16      0.20       276
     neutral       0.37      0.29      0.33       507

    accuracy                           0.43      1893
   macro avg       0.37      0.36      0.36      1893
weighted avg       0.42      0.43      0.42      1893

0.543629507138824


In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(smote_tomek_train_x, smote_tomek_train_y)

y_true = smote_tomek_test_y
y_pred = gnb.predict(smote_tomek_test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.23      0.41      0.29       290
       happy       0.57      0.54      0.56       820
         sad       0.29      0.14      0.19       276
     neutral       0.34      0.31      0.33       507

    accuracy                           0.40      1893
   macro avg       0.36      0.35      0.34      1893
weighted avg       0.42      0.40      0.40      1893

0.5386932770473143


# Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_x, train_y)

y_true = test_y
y_pred = gnb.predict(test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.23      0.41      0.29       290
       happy       0.57      0.54      0.56       820
         sad       0.29      0.14      0.19       276
     neutral       0.34      0.31      0.33       507

    accuracy                           0.40      1893
   macro avg       0.36      0.35      0.34      1893
weighted avg       0.42      0.40      0.40      1893

0.5386932770473143


In [20]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(smote_tomek_train_x, smote_tomek_train_y)

y_true = smote_tomek_test_y
y_pred = gnb.predict(smote_tomek_test_x)

target_names = ['angry', 'happy', 'sad', 'neutral']

print(classification_report(y_true, y_pred, target_names=target_names))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=2)
print(metrics.auc(fpr, tpr))

              precision    recall  f1-score   support

       angry       0.23      0.41      0.29       290
       happy       0.57      0.54      0.56       820
         sad       0.29      0.14      0.19       276
     neutral       0.34      0.31      0.33       507

    accuracy                           0.40      1893
   macro avg       0.36      0.35      0.34      1893
weighted avg       0.42      0.40      0.40      1893

0.5386932770473143
